In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, DocType, Date, Integer, Keyword, Text
from elasticsearch_dsl.connections import connections
import pandas as pd
from pandasticsearch import DataFrame
from datetime import datetime
import requests
import json

# Define a default Elasticsearch client
url = 'http://ec2-52-91-189-234.compute-1.amazonaws.com:9200'
ES_HOST = {"host" : "ec2-52-91-189-234.compute-1.amazonaws.com", "port" : 9200}
es = Elasticsearch(url)


In [2]:
FILE_URL = "http://apps.sloanahrens.com/qbox-blog-resources/kaggle-titanic-data/test.csv"
INDEX_NAME = 'titanic'
TYPE_NAME = 'passenger'
ID_FIELD = 'passengerid'

In [3]:
df = pd.read_csv(FILE_URL) 
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
es.indices.delete(index=INDEX_NAME, ignore=[400, 404])

In [6]:
#Import JSON to ES - SLOW (one by one)
tmp = df.to_json(orient = "records")
df_json= json.loads(tmp)
for doc in df_json:
    es.index(index=INDEX_NAME, doc_type=TYPE_NAME, body=doc, id=doc['PassengerId'])
    


PUT http://ec2-52-91-189-234.compute-1.amazonaws.com:9200/titanic/passenger/1268 [status:N/A request:10.041s]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.5/site-packages/urllib3/connectionpool.py", line 386, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/opt/conda/lib/python3.5/site-packages/urllib3/connectionpool.py", line 382, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/lib/python3.5/http/client.py", line 1197, in getresponse
    response.begin()
  File "/opt/conda/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/conda/lib/python3.5/socket.py", line 575, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception, another exception 

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='ec2-52-91-189-234.compute-1.amazonaws.com', port=9200): Read timed out. (read timeout=10))

In [ ]:
# list all the indexes
indices=es.indices.get_alias().keys()
sorted(indices)

In [ ]:
# save match all query as python variable
myquery={"query": {"match_all": {}}}

# execute the query using body parameter and return total number of records
# select count(*) from table
res = es.search(index=INDEX_NAME, body=myquery)
print("Got %d Hits:" % res['hits']['total'])


In [ ]:
# The same as above, variable myquery replaced by query string
res = es.search(index=INDEX_NAME, body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total'])

In [ ]:
# select count(*) from table where req_uuid = '940...33fa'
res = es.search(index=INDEX_NAME, request_timeout=60, body={"query": {"match":{"Sex":"female"}}})
mylist=res['hits']['hits']
print("Got %d Hits:" % res['hits']['total'])
print("Got %d Length:" % len(res['hits']['hits']))

In [ ]:
# show the column 'message' returned from above query
for i in range(len(res['hits']['hits'])):
    print(res['hits']['hits'][i]['_source']['Name'])

In [ ]:
#export elastic query results to pandas dataframe:
import pandas as pd
dfnew=pd.DataFrame(mylist)
dfnew